<a href="https://colab.research.google.com/github/R-Ramana/EE4211-Project/blob/main/EE4211_Group_9_Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EE4211 Group 9 Question 1

We use [this](https://datamall.lta.gov.sg/content/dam/datamall/datasets/LTA_DataMall_API_User_Guide.pdf) User Guide to get information on the API

In [ ]:
import requests
import json
import pandas as pd

# Change these values for different dates
year = "2022"
month = "4"
day = "12"
hour = "12"
minute = "30"
second = "0"

site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second.zfill(2)}'
print(site)
response_API = requests.get(site)
data = response_API.text
data = json.loads(data)
timestamp = data["items"][0]["timestamp"]
print(timestamp)

data = data["items"][0]["carpark_data"]
with open("EE4211data.json", 'w') as fp:
    json.dump(data, fp)
df = pd.read_json("EE4211data.json")
for heading in ("total_lots","lot_type","lots_available"):
    df[heading] = df["carpark_info"].apply(lambda x: x[0][heading])
df = df.drop(["carpark_info"], axis=1)
df

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A00
2022-04-12T12:29:27+08:00


,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-04-12T12:12:32,105,C,0
1,HLM,2022-04-12T12:12:42,583,C,0
2,RHM,2022-04-12T12:12:32,329,C,106
3,BM29,2022-04-12T12:12:32,97,C,1
4,Q81,2022-04-12T12:12:32,96,C,0
...,...,...,...,...,...
1961,B65M,2022-04-12T12:11:58,679,C,499
1962,B65L,2022-04-12T12:11:58,5,C,0
1963,BE3,2022-04-12T11:30:52,306,C,148
1964,BE3R,2022-04-12T11:30:52,195,C,150


1.1 Lot Type refers to the type of vehicles that the parking lot is meant for. C refers to car lot; H for heavy vehicles; Y for motorcycles.

1.2 

In [ ]:
second = "1"

site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second.zfill(2)}'
print(site)
response_API = requests.get(site)
data = response_API.text
data = json.loads(data)
timestamp = data["items"][0]["timestamp"]
print(timestamp)

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-04-12T12%3A30%3A01
2022-04-12T12:29:27+08:00


In [ ]:
minute = 30
second = 27

def print_timestamp(sec_str, min_str):
  site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{min_str.zfill(2)}%3A{sec_str.zfill(2)}'
  # print(site)
  response_API = requests.get(site)
  data = response_API.text
  data = json.loads(data)
  timestamp = data["items"][0]["timestamp"]
  print(timestamp)
  
  
  
for i in range (0, 90): 
  if second == 59:
    second = 0
    minute += 1
  else:
    second += 1

  min_str = str(minute)
  sec_str = str(second)
  print("Min: " + min_str + ", Sec: " + sec_str)
  print_timestamp(sec_str, min_str)


Min: 30, Sec: 28
2022-04-12T12:30:27+08:00
Min: 30, Sec: 29
2022-04-12T12:30:27+08:00
Min: 30, Sec: 30
2022-04-12T12:30:27+08:00
Min: 30, Sec: 31
2022-04-12T12:30:27+08:00
Min: 30, Sec: 32
2022-04-12T12:30:27+08:00
Min: 30, Sec: 33
2022-04-12T12:30:27+08:00
Min: 30, Sec: 34
2022-04-12T12:30:27+08:00
Min: 30, Sec: 35
2022-04-12T12:30:27+08:00
Min: 30, Sec: 36
2022-04-12T12:30:27+08:00
Min: 30, Sec: 37
2022-04-12T12:30:27+08:00
Min: 30, Sec: 38
2022-04-12T12:30:27+08:00
Min: 30, Sec: 39
2022-04-12T12:30:27+08:00
Min: 30, Sec: 40
2022-04-12T12:30:27+08:00
Min: 30, Sec: 41
2022-04-12T12:30:27+08:00
Min: 30, Sec: 42
2022-04-12T12:30:27+08:00
Min: 30, Sec: 43
2022-04-12T12:30:27+08:00
Min: 30, Sec: 44
2022-04-12T12:30:27+08:00
Min: 30, Sec: 45
2022-04-12T12:30:27+08:00
Min: 30, Sec: 46
2022-04-12T12:30:27+08:00
Min: 30, Sec: 47
2022-04-12T12:30:27+08:00
Min: 30, Sec: 48
2022-04-12T12:30:27+08:00
Min: 30, Sec: 49
2022-04-12T12:30:27+08:00
Min: 30, Sec: 50
2022-04-12T12:30:27+08:00
Min: 30, Se

1.3 (i) There are 2182 carparks in total included in the data.gov.sg car park database. The number of carparks stored in the API carpark database varies with time. The database has 1966 rows of data on our chosen date and time 2022-04-12T12:12:30. 

(ii) As time varies, we can observe that the number of carparks also varies as seen in the number of rows of data in the database. For example, at date and time 2022-04-13T12:12:30, there are only 1964 carparks included in the database.

(iii) By observing the latest 'update_datetime' field, we can identify carpark_number TJ37 (index 246) that has an anomalous sensor that has not reported its data since 2019-12-01T08:28:01, which is significantly far from the query-date-time. In addition, for some carparks such as carpark SK24, the data retrieved from the API includes 3 different rows of information for the same carpark showing different number of lots and different types of lots, with carpark SK24 showing it has 181 Y lots in one entry at 2016-02-19T11:19:28 and 181 H lots in another entry at 2016-02-19T11:19:29. The anomalous sensors at this carpark also shows more lots available (200) than total lots (181).


1.4 Generate hourly readings from the raw data. Select a one month interval and plot the hourly data (time-series) for that interval (aggregate results instead of plotting for each location individually). Identify any patterns in the visualization. Note: You will have to decide what to do if there are no carpark readings for a certain hour, for example, should you impute the missing data or ignore it.

In [ ]:
import requests
import json
import pandas as pd

# Change these values for different dates
year = "2022"
month = "2"
# day = "12"
# hour = "12"
# minute = "30"
second = 28
second_str = str(second)

# site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{minute.zfill(2)}%3A{second_str.zfill(2)}'
# print(site)
# response_API = requests.get(site)
# data = response_API.text
# data = json.loads(data)
# timestamp = data["items"][0]["timestamp"]
# print(timestamp)

feb_data = pd.DataFrame()

for day in range(1, 29):
    day_str = str(day)
    for hour in range(0, 24):
        hour_str = str(hour)
        for minute in range(0, 60, 20):
            minute_str = str(minute)
            site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day_str.zfill(2)}T{hour_str.zfill(2)}%3A{minute_str.zfill(2)}%3A{second_str.zfill(2)}'
            print(site)
            response_API = requests.get(site)
            data = response_API.text
            data = json.loads(data)
            if(not "items" in data):
                continue
            timestamp = data["items"][0]["timestamp"]
            print(timestamp)

            data = data["items"][0]["carpark_data"]
            with open("EE4211data.json", 'w') as fp:
                json.dump(data, fp)
            dt = (pd.read_json("EE4211data.json"))
            for heading in ("total_lots","lot_type","lots_available"):
                dt[heading] = dt["carpark_info"].apply(lambda x: x[0][heading])
            dt = dt.drop(["carpark_info"], axis=1)
            feb_data = feb_data.append(dt)
# feb_data.to_csv("feb_data_Q1-4", index = False)
feb_data

https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T00%3A00%3A28
2022-02-01T00:00:26+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T00%3A20%3A28
2022-02-01T00:20:27+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T00%3A40%3A28
2022-02-01T00:40:27+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T01%3A00%3A28
2022-02-01T01:00:26+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T01%3A20%3A28
2022-02-01T01:20:26+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T01%3A40%3A28
2022-02-01T01:40:27+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T02%3A00%3A28
2022-02-01T02:00:27+08:00
https://api.data.gov.sg/v1/transport/carpark-availability?date_time=2022-02-01T02%3A20%3A28
2022-02-01T02:20:27+08:00
https://api.data.gov.sg/v1/transport/carpark-availabilit

,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-01-31T23:51:30,105,C,56
1,HLM,2022-01-31T23:51:33,583,C,462
2,RHM,2022-01-31T23:51:30,329,C,129
3,BM29,2022-01-31T23:51:33,97,C,50
4,Q81,2022-01-31T23:51:36,96,C,75
...,...,...,...,...,...
1959,B65M,2022-02-28T23:16:31,665,C,380
1960,B65L,2022-02-28T23:16:31,6,C,0
1961,BE3,2022-02-28T22:55:46,306,C,53
1962,BE3R,2022-02-28T22:55:46,195,C,103


In [ ]:
feb_data_loaded = pd.read_csv("feb_data_Q1-4")
feb_data_loaded.head()

,carpark_number,update_datetime,total_lots,lot_type,lots_available
0,HE12,2022-01-31T23:51:30,105,C,56
1,HLM,2022-01-31T23:51:33,583,C,462
2,RHM,2022-01-31T23:51:30,329,C,129
3,BM29,2022-01-31T23:51:33,97,C,50
4,Q81,2022-01-31T23:51:36,96,C,75


In [ ]:
feb_data_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899288 entries, 0 to 3899287
Data columns (total 5 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   carpark_number   object
 1   update_datetime  object
 2   total_lots       int64 
 3   lot_type         object
 4   lots_available   int64 
dtypes: int64(2), object(3)
memory usage: 148.7+ MB


In [ ]:
import datetime
feb_data_loaded["update_datetime"] = pd.to_datetime(feb_data_loaded["update_datetime"])
feb_data_loaded['date'] = feb_data_loaded['update_datetime'].dt.date
feb_data_new = feb_data_loaded.loc[feb_data_loaded['date'] >= datetime.date(2022, 2, 1)]
feb_data_new.groupby('date').groups.keys()                                

dict_keys([datetime.date(2022, 2, 1), datetime.date(2022, 2, 2), datetime.date(2022, 2, 3), datetime.date(2022, 2, 4), datetime.date(2022, 2, 5), datetime.date(2022, 2, 6), datetime.date(2022, 2, 7), datetime.date(2022, 2, 8), datetime.date(2022, 2, 9), datetime.date(2022, 2, 10), datetime.date(2022, 2, 11), datetime.date(2022, 2, 12), datetime.date(2022, 2, 13), datetime.date(2022, 2, 14), datetime.date(2022, 2, 15), datetime.date(2022, 2, 16), datetime.date(2022, 2, 17), datetime.date(2022, 2, 18), datetime.date(2022, 2, 19), datetime.date(2022, 2, 20), datetime.date(2022, 2, 21), datetime.date(2022, 2, 22), datetime.date(2022, 2, 23), datetime.date(2022, 2, 24), datetime.date(2022, 2, 25), datetime.date(2022, 2, 26), datetime.date(2022, 2, 27), datetime.date(2022, 2, 28)])

In [ ]:
feb_data_new["hour"] = feb_data_new["update_datetime"].dt.hour
feb_data_new["day"] = feb_data_new["update_datetime"].dt.day
feb_data_new["percent_lots_filled"] = (feb_data_new["lots_available"]/feb_data_new["total_lots"]) * 100
feb_data_new.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,carpark_number,update_datetime,total_lots,lot_type,lots_available,date,hour,day,percent_lots_filled
3928,HE12,2022-02-01 00:23:00,105,C,59,2022-02-01,0,1,56.190476
3929,HLM,2022-02-01 00:23:03,583,C,465,2022-02-01,0,1,79.759863
3930,RHM,2022-02-01 00:23:00,329,C,125,2022-02-01,0,1,37.993921
3931,BM29,2022-02-01 00:23:03,97,C,50,2022-02-01,0,1,51.546392
3932,Q81,2022-02-01 00:23:06,96,C,73,2022-02-01,0,1,76.041667


In [ ]:
hourly_data = feb_data_new.groupby('hour', as_index = False)['percent_lots_filled'].mean()
hourly_data

,hour,percent_lots_filled
0,0,40.025986
1,1,39.477097
2,2,39.196754
3,3,38.913130
4,4,38.477010
5,5,38.773376
6,6,40.580233
7,7,44.818625
8,8,48.089163
9,9,50.241961
